In [ ]:
# Pix2Pix GAN
# Importing required libraries
import tensorflow as tf
from tensorflow.keras.layers import Input, Concatenate, Conv2D, Conv2DTranspose, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dense
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Reshape
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dropout, Add
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

# Define image dimensions and other hyperparameters
img_rows = 256
img_cols = 256
channels = 3
img_shape = (img_rows, img_cols, channels)
latent_dim = 100
num_classes = 10
batch_size = 32
epochs = 600
sample_interval = 50



Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 35ms/step
Epoch 54/600 | Batch 2/5 | D Loss: [3.87455503e-14 1.00000000e+00] | G Loss: 3.052591864616261e-06
1/1 [==============================] - 0s 34ms/step
Epoch 54/600 | Batch 3/5 | D Loss: [4.8088418e-12 1.0000000e+00] | G Loss: 0.10846056789159775
1/1 [==============================] - 0s 33ms/step
Epoch 54/600 | Batch 4/5 | D Loss: [5.07855767e-07 1.00000000e+00] | G Loss: 0.17278479039669037
1/1 [==============================] - 0s 36ms/step
Epoch 54/600 | Batch 5/5 | D Loss: [0.10324489 0.96875   ] | G Loss: 2.9023358821868896
1/1 [==============================] - 0s 31ms/step
Epoch 55/600 | Batch 1/5 | D Loss: [0.00850122 1.        ] | G Loss: 1.1983416080474854
1/1 [==============================] - 0s 31ms/step
Epoch 55/600 | Batch 2/5 | D Loss: [4.04975078e-06 1.00000000e+00] | G Loss: 0.30098021030426025
1/1 [==============================] - 0s 35ms/step
Epoch 55/600 | Batch 

In [ ]:
# Define directories for images and sketches
img_dir = 'cuhk_training'
sketch_dir = 'cuhk_train_sketch'

# Load and preprocess images and sketches
def load_data():
    # Sorting the data
    img_list = sorted(os.listdir(img_dir))
    sketch_list = sorted(os.listdir(sketch_dir))
    img_list.sort()
    sketch_list.sort()
    num_imgs = len(img_list)
    imgs = np.zeros((num_imgs, img_rows, img_cols, channels), dtype=np.float32)
    sketches = np.zeros((num_imgs, img_rows, img_cols, channels), dtype=np.float32)
    # Looping over the image folders
    for i in range(num_imgs):
        img_path = os.path.join(img_dir, img_list[i])
        sketch_path = os.path.join(sketch_dir, sketch_list[i])
        # img = load_img(img_path, target_size=img_shape)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (img_rows, img_cols))
        # Preprocessing: Apply Gaussian blur to input images to remove noise
        img = cv2.GaussianBlur(img, (5,5), 0)
        
        sketch = cv2.imread(sketch_path, cv2.IMREAD_GRAYSCALE)
        sketch = cv2.resize(sketch, (img_rows, img_cols))
        # sketch = load_img(sketch_path, target_size=img_shape, color_mode='grayscale')
        img = img_to_array(img)
        sketch = img_to_array(sketch)
        img = (img / 127.5) - 1.0
        sketch = (sketch / 127.5) - 1.0
        imgs[i] = img
        sketches[i] = sketch
    return imgs, sketches



In [ ]:
# Load pre-trained VGG19 model
vgg = VGG19(include_top=False, weights='imagenet', input_shape=img_shape)
vgg.trainable = False

# Get the output of the fourth convolutional block in VGG19
vgg_output = vgg.layers[8].output

# Create a new model that outputs the output of the fourth convolutional block
vgg_model = Model(inputs=vgg.input, outputs=vgg_output)

# Define the perceptual loss function
def perceptual_loss(y_true, y_pred):
    # Calculate the mean squared error between the true and predicted images
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    
    # Get the VGG19 features of the true and predicted images
    true_features = vgg_model(y_true)
    pred_features = vgg_model(y_pred)
    
    # Calculate the mean squared error between the VGG19 features of the true and predicted images
    feature_loss = tf.keras.losses.mean_squared_error(true_features, pred_features)
    
    # Return the sum of the mean squared error between the true and predicted images and the mean squared error between the VGG19 features of the true and predicted images
    return mse_loss + feature_loss



In [ ]:
# Defining the generator and discriminator architecture
def build_generator():
    # Encoder
    img_shape = (256, 256, channels)
    input_img = Input(shape=img_shape)
    encoder_layers = [
        Conv2D(64, (4, 4), strides=(2, 2), padding='same', activation='relu', input_shape=img_shape),
        Conv2D(128, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2D(256, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2D(512, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2D(512, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2D(512, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2D(512, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2D(512, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization()
    ]
    encoded = input_img
    for layer in encoder_layers:
        encoded = layer(encoded)

    # Decoder
    decoder_layers = [
        Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', activation='relu', use_bias=False),
        BatchNormalization(),
        Conv2DTranspose(channels, (4, 4), strides=(2, 2), padding='same', activation='tanh')
    ]
    decoded = encoded
    for layer in decoder_layers:
        decoded = layer(decoded)

    generator = Model(inputs=input_img, outputs=decoded)
    optimizer = Adam(lr=0.0002, beta_1=0.5)
    generator.compile(loss='perceptual_loss', optimizer=optimizer)
    return generator


# Defining the discriminator architecture
def build_discriminator(img_shape):
    real_img = Input(shape=img_shape)
    generated_img = Input(shape=img_shape)

    x = Concatenate()([real_img, generated_img])
    x = Conv2D(32, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv2D(64, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)

    discriminator = Model(inputs=[real_img, generated_img], outputs=x)
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['binary_accuracy'])
    return discriminator


In [ ]:

# Load and preprocess data
imgs, sketches = load_data()

# Build generator and discriminator
generator = build_generator()
discriminator = build_discriminator(img_shape)

# Build GAN
discriminator.trainable = False
gan_input = Input(shape=img_shape)
gan_output = discriminator([generator(gan_input), gan_input])
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

# Define labels for real and fake images
real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))



In [ ]:
D_loss = []
G_loss = []
# Training loop
for epoch in range(epochs):
    for i in range(len(imgs) // batch_size):
        # Train discriminator
        idx = np.random.randint(0, len(imgs), batch_size)
        real_imgs = imgs[idx]
        sketch_imgs = sketches[idx]
        
        fake_imgs = generator.predict(sketch_imgs)
        
        d_loss_real = discriminator.train_on_batch([real_imgs, sketch_imgs], [real])
        d_loss_fake = discriminator.train_on_batch([fake_imgs, sketch_imgs], [fake])
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        idx = np.random.randint(0, len(imgs), batch_size)
        sketch_imgs = sketches[idx]
        g_loss = gan.train_on_batch(sketch_imgs, real)

        # Print progress
        print(f"Epoch {epoch+1}/{epochs} | Batch {i+1}/{len(imgs)//batch_size} | D Loss: {d_loss} | G Loss: {g_loss}")
        D_loss.append(d_loss)
        G_loss.append(g_loss)


psnr_scores = []
ssim_scores = []

# Calculating PSNR and SSIM scores
for i in range(len(imgs) // batch_size):
    idx = np.random.randint(0, len(imgs), batch_size)
    real_imgs = imgs[idx]
    sketch_imgs = sketches[idx]

    fake_imgs = generator.predict(sketch_imgs)

    for j in range(len(real_imgs)):
        psnr_scores.append(peak_signal_noise_ratio(real_imgs[j], fake_imgs[j]))
        ssim_scores.append(structural_similarity(real_imgs[j], fake_imgs[j], multichannel=True))

print(f"Average PSNR: {np.mean(psnr_scores)} | Average SSIM: {np.mean(ssim_scores)}")

# Plot the PSNR and SSIM scores
plt.hist(psnr_scores, bins=20, alpha=0.5, label="PSNR")
plt.hist(ssim_scores, bins=20, alpha=0.5, label="SSIM")
plt.legend()
plt.show()
